# Phase 1: Acquire Data

This notebook will get data about your Azure Databricks VM usage.  It looks at the last 12 days of the
[Azure Activity Logs](https://learn.microsoft.com/en-us/azure/azure-monitor/essentials/activity-log) for the
Databricks managed resource groups to determine what VMs are created and how long they are used.

Through this process, we will build a tiny little lakehouse to store the Databricks VM usage data in its
various stages of processing.  The Azure Activity Log data is saved in its raw form as JSON data.
This notebook then uses Spark to transform that raw data into Delta files in a Bronze, Silver, and Gold
layer.  Each of these steps further transforms the data and prepares it for analysis.

The Azure Activity Log can be accessed through the Azure Management API.  Rather than making raw REST calls, we'll use the 
[Azure Libraries for Python](https://learn.microsoft.com/en-us/azure/developer/python/sdk/azure-sdk-overview).  We can use the `%pip`
magic command to install them in our current Python session.

In [0]:
%pip install azure-identity azure-mgmt-monitor azure-mgmt-resourcegraph azure-mgmt-resource azure-mgmt-compute

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.7/164.7 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 92.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.4/192.4 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.7/97.7 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 10.6 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.4.0
    Not uninstalling typing-extensions at /databricks/python3/lib/python3.10/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-1

## Configuration

These are the configuration settings you will need to adjust to specify how the data acquisition process should operate.

**`service_principal`** contains the information about the service principal that you will use to connect to the Azure Management API.
If you used the `az ad sp create-for-rbac` CLI command to create the service principal, then you can just straight copy and paste the
output of that command here.

**`subscription_ids`** is a Python list of the subscription ID's for which you want to gather Databricks VM usage data.  If you leave
the list empty, then this process will pull Databricks VM usage data for all subscriptions to which the service principal has access.

**`delta_location`** is the path where the Databricks VM usage data will be saved.  It can be a location on the DBFS or it can use
any standard HDFS path (*e.g.* an `abfss://` path)

**`days_to_pull`** specifies how many days into the past the Azure Activity Log should be queried.  By default, the Azure Activity Log
retains 90 days of data.  Specifying a lower number will speed up the process because there will be fewer queries made.  However,
you will want to pull several days of data to get a clearer picture of the usage patterns.

**`query_window_hours`** controls how many hours will be covered by each query to the Azure Activity Log.  Instead of issuing one, big
query to the Activity Log to pull all of the history at once, this notebook will break it up into multiple queries.  If you don't have
a lot of Databricks VM activity, you can set this number higher.  For example, setting it to "24" will issue one query per day.  If
you have a high volume of Databricks VM activity, set this number lower.  This will break the querying process into smaller, more
reliable chunks.  If you're not sure, just leave it at 4.

In [0]:
service_principal = {
  "appId": "<your App ID / Client ID (should be a guid)>",
  "displayName": "<display name for your SP (optional)>",
  "password": "<Password / Secret (probably a 34-byte string)>",
  "tenant": "<your Tenant ID (should be a guid)>"
}

subscription_ids = []

delta_location = "dbfs:/vm-observation"

days_to_pull = 12
query_window_hours = 4

## Raw Data

Read the Azure Activity Logs and save the results as JSON files.

In [0]:
import azure.mgmt.resourcegraph as arg
from datetime import datetime, timedelta
from azure.identity import ClientSecretCredential, AzureCliCredential
from azure.mgmt.resource import SubscriptionClient
from azure.mgmt.monitor import MonitorManagementClient


credential = ClientSecretCredential(service_principal["tenant"], service_principal["appId"], service_principal["password"])
#credential = AzureCliCredential() # (Optional... if you want to use Azure CLI to login instead of using a service principal)

while delta_location.endswith("/"):
    delta_location = delta_location[:-1]

# If we were not given a list of subscriptions, just get all of the ones the SP has access to
if not subscription_ids:
    subscriptions_client = SubscriptionClient(credential)
    subscription_ids = [s.as_dict()['subscription_id'] for s in subscriptions_client.subscriptions.list()]


def get_adbx_workspaces():
    arg_client = arg.ResourceGraphClient(credential)
    query_options = arg.models.QueryRequestOptions(result_format = "objectArray")

    query_string = 'resources | where type == "microsoft.databricks/workspaces" | project workspaceId = id, name, subscriptionId, sku = sku.name, managedResourceGroupId = properties.managedResourceGroupId, location'
    query_request = arg.models.QueryRequest(subscriptions=subscription_ids, query=query_string, options=query_options)
    results = arg_client.resources(query_request)

    output = results.as_dict()["data"]
    for o in output:
        o["managedResourceGroupName"] = o["managedResourceGroupId"].split('/')[-1]
        
        monitor_client = MonitorManagementClient(credential, o["subscriptionId"])
        diagnostic_settings = monitor_client.diagnostic_settings.list(o["workspaceId"])
        o["diagnostic_settings"] = [s.as_dict() for s in diagnostic_settings]
    
    return output


def write_activity_log_for_workspace(workspace, date, i, json_filename):
    print(f"{i}  Getting activity log for workspace \"{workspace['name']}\"    {date.date()}  {date.hour}:00")
    monitor_client = MonitorManagementClient(credential, workspace["subscriptionId"])
    
    start_time = date - timedelta(hours=query_window_hours) + timedelta(microseconds=1)
    end_time = date
    filter = f"eventTimestamp ge '{start_time.isoformat()}' and eventTimestamp le '{end_time.isoformat()}' and resourceGroupName eq '{workspace['managedResourceGroupName']}' and resourceType eq 'Microsoft.Compute/virtualMachines'"
    
    log_entries = monitor_client.activity_logs.list(filter)
    logs = [i.as_dict() for i in log_entries]

    for log in logs:
        if "properties" in log:
            if "responseBody" in log["properties"]:
                response_body = json.loads(log["properties"]["responseBody"])
                log["properties"]["responseBody"] = response_body

    print(f"{i} Saving log to file: {json_filename}")
    dbutils.fs.put(json_filename, json.dumps({"workspace": workspace, "logs": logs, "capture_time": datetime.now().isoformat() + "Z"}))

In [0]:
import json

workspaces = get_adbx_workspaces()
dbutils.fs.put(
    f"{delta_location}/raw/workspaces/workspaces.json",
    json.dumps({"workspaces": workspaces, "capture_time": datetime.now().isoformat() + "Z"}),
    overwrite=True
)

ws_dict = {ws['managedResourceGroupName']: ws for ws in workspaces}
timestamp = datetime.now()

i = 0
param_sets = []

for rg_name in ws_dict:
    ws = ws_dict[rg_name]
    json_path = f"{delta_location}/raw/activity_logs/subscription={ws['subscriptionId']}/rg={rg_name}"

    for h in range(0, 24 * days_to_pull, query_window_hours):
        i = i + 1
        date = timestamp - timedelta(hours=h)

        param_sets.append({
            "workspace": ws,
            "date": date,
            "i": i,
            "json_filename": f"{json_path}/{date.date()}-{date.hour}.json"
        })

Wrote 1689 bytes.


This is where we actually do the work.  We use Python's multi-threading to query the Azure Activity Log API in parallel.

In [0]:
import concurrent.futures
import psutil

concurrent_thread_count = psutil.cpu_count()
print(f"\n{'*' * 80}\n")
print(f"Starting at {datetime.now()}")
print(f"Going to process {len(param_sets)} API queries, running {concurrent_thread_count} at a time")
print(f"\n{'*' * 80}\n")

with concurrent.futures.ThreadPoolExecutor(max_workers=concurrent_thread_count) as executor:
    future_to_logwriter = {executor.submit(write_activity_log_for_workspace, ps["workspace"], ps["date"], ps["i"], ps["json_filename"]): ps for ps in param_sets}
    for future in concurrent.futures.as_completed(future_to_logwriter):
        ps = future_to_logwriter[future]
        try:
            data = future.result()
        except Exception as exc:
            print(f"{ps['workspace']['managedResourceGroupName']} {ps['date']} generated an exception: {exc}")


********************************************************************************

Starting at 2023-11-09 15:27:29.191649
Going to process 144 API queries, running 16 at a time

********************************************************************************

1  Getting activity log for workspace "adbx_pool_sizer"    2023-11-09  15:00
2  Getting activity log for workspace "adbx_pool_sizer"    2023-11-09  11:00
3  Getting activity log for workspace "adbx_pool_sizer"    2023-11-09  7:00
4  Getting activity log for workspace "adbx_pool_sizer"    2023-11-09  3:00
5  Getting activity log for workspace "adbx_pool_sizer"    2023-11-08  23:00
6  Getting activity log for workspace "adbx_pool_sizer"    2023-11-08  19:00
7  Getting activity log for workspace "adbx_pool_sizer"    2023-11-08  15:00
8  Getting activity log for workspace "adbx_pool_sizer"    2023-11-08  11:00
9  Getting activity log for workspace "adbx_pool_sizer"    2023-11-08  7:00
10  Getting activity log for workspace "adbx_pool_

#### Get VM SKU Information
In addition to the VM usage logs, we also need to get some information about the VM SKU's.  We will pull it from the Azure API just to be sure
we always have the latest data.  The available VM types can vary by subscription and by region so we will have to query with multiple sets of
parameters.

In [0]:
from azure.mgmt.compute import ComputeManagementClient

sub_locs = [(ws['subscriptionId'], ws['location']) for ws in workspaces]
sub_locs = list(set(sub_locs))

for sub_loc in sub_locs:
    print(f"Getting VM SKUs for subscription {sub_loc[0]} in {sub_loc[1]} region")
    compute_client = ComputeManagementClient(credential, sub_loc[0])
    results = compute_client.virtual_machine_sizes.list(sub_loc[1])
    results = [i.as_dict() for i in results]

    for r in results:
        r["subscription_id"] = sub_loc[0]
        r["location"] = sub_loc[1]

    json_path = f"{delta_location}/raw/vm_skus/subscription={sub_loc[0]}"
    json_filename = f"{json_path}/{sub_loc[1]}.json"
    dbutils.fs.put(json_filename, json.dumps(results))

Getting VM SKUs for subscription 6de50f6f-dc7e-431d-96ea-53da23ade313 in eastus region
Wrote 194806 bytes.


## Bronze Layer

This is the first step in transforming our raw data.  We will simply read the JSON files and apply only the bare minimum of transformations
before converting it to Delta files.

In [0]:
import pyspark.sql.functions as F

spark.conf.set("spark.sql.caseSensitive", "true")

df = spark.read \
        .format("json") \
        .load(delta_location + "/raw/activity_logs") \
        .selectExpr("workspace.workspaceId", "workspace.name AS workspaceName", "workspace.sku AS workspaceSku", "workspace.subscriptionId",
                    "workspace.managedResourceGroupId", "CAST(capture_time AS TIMESTAMP) AS capture_time", "explode(logs) AS log_entry") \
        .selectExpr("CAST(log_entry.event_timestamp AS TIMESTAMP) AS event_timestamp", "lower(split(log_entry.resource_id, '/')[8]) as vm_id", "*") \
        .cache()

There are some columns that we definitely won't need for analysis and they have some challenging column names.  So we will make life
easier for us and just drop those columns.

In [0]:
claim_cols = df.select("log_entry.claims.*").columns

for col in claim_cols:
    if col.startswith("http") and ":/" in col:
        df = df.withColumn("log_entry", F.col("log_entry").dropFields("claims.`" + col + "`"))

df = df.withColumn("log_entry", F.col("log_entry").dropFields("properties.responseBody.identity"))

Each of the Azure tags applied to a Databricks VM is represented as its own field in a nested column.  However, Delta does not allow the names of two
columns to vary only by case.  For example, Delta won't let you have a column called `project_name` and `Project_Name` in the same table.  Across a large
organization, it's easy for Azure tags to have inconsistent casing in their names.  Therefore, as we convert the raw data to Delta, we have to find
Azure tag columns that have the same name but different letter casing and combine those into one column.

In [0]:
tagnames = df.select("log_entry.properties.responseBody.tags.*").columns

instance_pool_tagname = "DatabricksInstancePoolId"
if instance_pool_tagname not in tagnames:
    print("Instance Pool ID column not in JSON schema.  Adding it to Delta file.")
    df = df.withColumn("log_entry", F.col("log_entry").withField("properties.responseBody.tags." + instance_pool_tagname, F.lit(None).astype("string"))) 


tagname_groups = {}
for tagname in tagnames:
    tagname_groups.setdefault(tagname.lower(), []).append(tagname)

for key in tagname_groups:
    values = tagname_groups[key]
    if len(values) > 1:
        keep = values[0]
        for i in range(1, len(values)):
            print(f"combine tag \"{values[i]}\" into tag \"{keep}\"")
            df = df.withColumn("log_entry",
                F.col("log_entry").withField("properties.responseBody.tags." + keep,
                                                F.coalesce(
                                                    F.col("log_entry.properties.responseBody.tags." + keep),
                                                    F.col("log_entry.properties.responseBody.tags." + values[i])
                                                )
                                            )
            ) 
            df = df.withColumn("log_entry", F.col("log_entry").dropFields("properties.responseBody.tags." + values[i]))

Delta has some restrictions on what characters can be used in column names.  Most notabaly, this includes spaces.  We will look through
all of the Azure tag columns and change any invalid characters to underscores.

In [0]:
tagnames = df.select("log_entry.properties.responseBody.tags.*").columns
bad_chars = " ,;{}()\n\t="

for tagname in tagnames:
    for c in bad_chars:
        if c in tagname:
            new_tagname = tagname.replace(c, "_")
            print(f"Change tag \"{tagname}\" to \"{new_tagname}")

            if new_tagname in tagnames:
                df = df.withColumn("log_entry",
                    F.col("log_entry").withField("properties.responseBody.tags." + new_tagname,
                                                    F.coalesce(
                                                        F.col("log_entry.properties.responseBody.tags." + new_tagname),
                                                        F.col("log_entry.properties.responseBody.tags." + tagname)
                                                    )
                                                )
                ) 
            else:
                df = df.withColumn("log_entry",
                    F.col("log_entry").withField(
                        "properties.responseBody.tags." + new_tagname,
                        F.col("log_entry.properties.responseBody.tags." + tagname)
                    )
                )

            df = df.withColumn("log_entry", F.col("log_entry").dropFields("properties.responseBody.tags.`" + tagname + "`"))

In [0]:
df.write.format("delta").mode("overwrite").save(delta_location + "/bronze/activity_logs")

#### VM SKU Information
We also need to convert our VM SKU data from JSON to Delta!  This is much simpler, though, because the data is not complex.

In [0]:
df = spark.read \
        .format("json") \
        .load(delta_location + "/raw/vm_skus")

df.write.format("delta").mode("overwrite").save(delta_location + "/bronze/vm_skus")

## Silver Layer

Now that we have our data in the easy-to-use Delta format, we can begin transforming it into a format that is more useful for analysis.
The raw data contains a log of all of the VM operations that Azure performed in the Databricks-managed resource groups.  However many
of those operations aren't useful for our analysis.  We only want to know when a new VM was created or an existing VM was deleted.
(Though we won't use it for this analysis, we'll also record when spot instances are evicted because that could be interesting to
study.)

In [0]:
import pyspark.sql.functions as F

df = spark.read.format("delta").load(delta_location + "/bronze/activity_logs")

# Get rows for the creation, eviction, or deletion of a VM
activities = df \
                .filter("""
                    (log_entry.properties.responseBody IS NOT NULL AND log_entry.resource_type.value != 'Microsoft.Compute/virtualMachines/extensions')
                    OR lower(log_entry.operation_name.value) LIKE '%spot%'
                    OR (log_entry.operation_name.value == 'Microsoft.Compute/virtualMachines/delete' AND log_entry.event_name.value == 'EndRequest')
                """) \
                .selectExpr("vm_id", "event_timestamp", "log_entry.properties.statusCode", "log_entry.operation_name.value AS operation", "subscriptionId",
                            "workspaceName", "log_entry.properties.responseBody.properties.hardwareProfile.vmSize", "log_entry.properties.responseBody.properties.priority",
                            "log_entry.properties.responseBody.location", "log_entry.properties.responseBody.tags") \
                .orderBy("vm_id", "event_timestamp")

For our analysis, we need the Azure tags associated with the VM.  This tells us which job, cluster, or pool the VM was part of.  However, we only
get the Azure tag data when a VM is created. If a VM was started before the period of time we are studying, we won't have that tag data in our logs.
Therefore, we will just throw out any data for VM's that are missing this critical data.

In [0]:
# Remove all information about VMs if we don't have any tags for it
activities.createOrReplaceTempView("activities_table")

no_starts = spark.sql("SELECT vm_id, COUNT(*), MIN(event_timestamp), MAX(event_timestamp) FROM activities_table WHERE vm_id NOT IN (SELECT vm_id FROM activities_table WHERE tags IS NOT NULL) GROUP BY vm_id ORDER BY 3 DESC")
print(f"\nRemoving data for {no_starts.count():,} VMs because there is no creation data in the logs.\n")
no_starts.show(truncate=False)

activities = spark.sql("SELECT * FROM activities_table WHERE vm_id IN (SELECT vm_id FROM activities_table WHERE tags IS NOT NULL)")


Removing data for 0 VMs because there is no creation data in the logs.

+-----+--------+--------------------+--------------------+
|vm_id|count(1)|min(event_timestamp)|max(event_timestamp)|
+-----+--------+--------------------+--------------------+
+-----+--------+--------------------+--------------------+



Since not all data points (like Azure tag data) is present in every log entry, we will order the logs by VM ID and timestamp.  We will carry forward
these data points to all future rows that are missing the data.

That is to say, if we have sparse data like this...

| VM ID | Timestamp | VM Size  |
|-------|-----------|----------|
| 1234  | 13:38:34  | E8ds_v4  |
| 1234  | 13:42:18  | *null*   |
| 1234  | 13:46:12  | *null*   |
| 1234  | 13:47:09  | *null*   |
| 1234  | 13:49:58  | E16ds_v4 |
| 1234  | 13:50:02  | *null*   |
| 5678  | 12:17:24  | DS3_v3   |
| 5678  | 12:22:32  | *null*   |
| 5678  | 12:34:02  | *null*   |

... we will carry forward the values in the sparse column so it looks like this:

| VM ID | Timestamp | VM Size  |
|-------|-----------|----------|
| 1234  | 13:38:34  | E8ds_v4  |
| 1234  | 13:42:18  | E8ds_v4  |
| 1234  | 13:46:12  | E8ds_v4  |
| 1234  | 13:47:09  | E8ds_v4  |
| 1234  | 13:49:58  | E16ds_v4 |
| 1234  | 13:50:02  | E16ds_v4 |
| 5678  | 12:17:24  | DS3_v3   |
| 5678  | 12:22:32  | DS3_v3   |
| 5678  | 12:34:02  | DS3_v3   |

In [0]:
from pyspark.sql.window import Window

window_spec = Window.partitionBy("vm_id").orderBy("event_timestamp")

activities = activities \
                .withColumn("activity_number", F.row_number().over(window_spec)) \
                .withColumn("event_end", F.lead("event_timestamp").over(window_spec) - F.expr('INTERVAL 1 MICROSECOND')) \
                .withColumn("vmSize", F.last("vmSize", True).over(window_spec)) \
                .withColumn("priority", F.last("priority", True).over(window_spec)) \
                .withColumn("location", F.last("location", True).over(window_spec)) \
                .withColumn("tags", F.last("tags", True).over(window_spec))

print(f"Rows: {activities.count():,}")

Rows: 4,594


Silver processing complete!  Write out this version of the data to its own Delta table.

In [0]:
activities.write.format("delta").mode("overwrite").save(delta_location + "/silver/activities")

#### VM SKU Information
This will be the final step in processing the VM SKU data.  We will interpret the SKU name to get additional information about the VM series.

For information on how the VM SKU's are named, see this document: [Azure virtual machine sizes naming conventions](https://learn.microsoft.com/en-us/azure/virtual-machines/vm-naming-conventions)

In [0]:
df = spark.read.format("delta").load(delta_location + "/bronze/vm_skus")

df = df \
        .drop("subscription") \
        .withColumn("memory_in_gb", F.expr("memory_in_mb / 1024")) \
        .withColumn("name_without_version", F.expr("REGEXP_REPLACE(name, '\_v[0-9]+', '')")) \
        .withColumn("name_parts", F.split("name", "_")) \
        .withColumn("is_promo", F.expr("name LIKE '%_Promo'")) \
        .withColumn("family_group", F.expr("name_parts[0]")) \
        .withColumn("family", F.expr("REGEXP_EXTRACT(name_parts[1], '^[A-z]+', 0)")) \
        .withColumn("name_number", F.expr("REGEXP_EXTRACT(name_parts[1], '^([A-z]+)([0-9]+)', 2)")) \
        .withColumn("constrained_vcpu", F.expr("REGEXP_EXTRACT(name_parts[1], '^([A-z]+)([0-9]+)-([0-9]+)', 3)")) \
        .withColumn("additive_features", F.expr("REGEXP_EXTRACT(name_parts[1], '^([A-z]+)([0-9]+)((-[0-9]+)?)(.*)', 5)")) \
        .withColumn("accelerator_type", F.expr("CASE WHEN name_parts[2] != 'Promo' AND name_parts[2] NOT RLIKE 'v[0-9]+' THEN name_parts[2] ELSE NULL END")) \
        .withColumn("version_number", F.expr("CASE WHEN name_parts[2] RLIKE 'v[0-9]+' THEN name_parts[2] ELSE CASE WHEN name_parts[3] RLIKE 'v[0-9]+' THEN name_parts[3] ELSE 'v1' END END")) \
        .drop("name_parts")

df = df \
        .withColumn("is_amd", F.expr("additive_features RLIKE 'a'")) \
        .withColumn("is_block_storage_performance", F.expr("additive_features RLIKE 'b'")) \
        .withColumn("has_local_temp_disk", F.expr("additive_features RLIKE 'd'")) \
        .withColumn("is_isolated", F.expr("additive_features RLIKE 'i'")) \
        .withColumn("is_low_memory", F.expr("additive_features RLIKE 'l'")) \
        .withColumn("is_memory_intensive", F.expr("additive_features RLIKE 'm'")) \
        .withColumn("is_arm", F.expr("additive_features RLIKE 'p'")) \
        .withColumn("is_tiny_memory", F.expr("additive_features RLIKE 't'")) \
        .withColumn("has_premium_storage", F.expr("additive_features RLIKE 's'")) \
        .withColumn("is_confidential", F.expr("additive_features RLIKE 'C'")) \
        .withColumn("is_node_packing", F.expr("additive_features RLIKE 'NP'"))

display(df)

location,max_data_disk_count,memory_in_mb,name,number_of_cores,os_disk_size_in_mb,resource_disk_size_in_mb,subscription_id,memory_in_gb,name_without_version,is_promo,family_group,family,name_number,constrained_vcpu,additive_features,accelerator_type,version_number,is_amd,is_block_storage_performance,has_local_temp_disk,is_isolated,is_low_memory,is_memory_intensive,is_arm,is_tiny_memory,has_premium_storage,is_confidential,is_node_packing
eastus,2,512,Standard_B1ls,1,1047552,4096,6de50f6f-dc7e-431d-96ea-53da23ade313,0.5,Standard_B1ls,false,Standard,B,1,,ls,null,v1,false,false,false,false,true,false,false,false,true,false,false
eastus,2,2048,Standard_B1ms,1,1047552,4096,6de50f6f-dc7e-431d-96ea-53da23ade313,2.0,Standard_B1ms,false,Standard,B,1,,ms,null,v1,false,false,false,false,false,true,false,false,true,false,false
eastus,2,1024,Standard_B1s,1,1047552,4096,6de50f6f-dc7e-431d-96ea-53da23ade313,1.0,Standard_B1s,false,Standard,B,1,,s,null,v1,false,false,false,false,false,false,false,false,true,false,false
eastus,4,8192,Standard_B2ms,2,1047552,16384,6de50f6f-dc7e-431d-96ea-53da23ade313,8.0,Standard_B2ms,false,Standard,B,2,,ms,null,v1,false,false,false,false,false,true,false,false,true,false,false
eastus,4,4096,Standard_B2s,2,1047552,8192,6de50f6f-dc7e-431d-96ea-53da23ade313,4.0,Standard_B2s,false,Standard,B,2,,s,null,v1,false,false,false,false,false,false,false,false,true,false,false
eastus,8,16384,Standard_B4ms,4,1047552,32768,6de50f6f-dc7e-431d-96ea-53da23ade313,16.0,Standard_B4ms,false,Standard,B,4,,ms,null,v1,false,false,false,false,false,true,false,false,true,false,false
eastus,16,32768,Standard_B8ms,8,1047552,65536,6de50f6f-dc7e-431d-96ea-53da23ade313,32.0,Standard_B8ms,false,Standard,B,8,,ms,null,v1,false,false,false,false,false,true,false,false,true,false,false
eastus,16,49152,Standard_B12ms,12,1047552,98304,6de50f6f-dc7e-431d-96ea-53da23ade313,48.0,Standard_B12ms,false,Standard,B,12,,ms,null,v1,false,false,false,false,false,true,false,false,true,false,false
eastus,32,65536,Standard_B16ms,16,1047552,131072,6de50f6f-dc7e-431d-96ea-53da23ade313,64.0,Standard_B16ms,false,Standard,B,16,,ms,null,v1,false,false,false,false,false,true,false,false,true,false,false
eastus,32,81920,Standard_B20ms,20,1047552,163840,6de50f6f-dc7e-431d-96ea-53da23ade313,80.0,Standard_B20ms,false,Standard,B,20,,ms,null,v1,false,false,false,false,false,true,false,false,true,false,false


The Azure API does not provide information about which series a VM type belongs to.  So we will copy some data from the docs and use that instead.
This may have to be manually updated from time to time.

In [0]:
# Copied from:  https://learn.microsoft.com/en-us/azure/virtual-machines/sizes

series_data = [
    ["General purpose",
     "B, Dsv3, Dv3, Dasv4, Dav4, DSv2, Dv2, Av2, DC, DCv2, Dpdsv5, Dpldsv5, Dpsv5, Dplsv5, Dv4, Dsv4, Ddv4, Ddsv4, Dv5, Dsv5, Ddv5, Ddsv5, Dasv5, Dadsv5",
     "Balanced CPU-to-memory ratio. Ideal for testing and development, small to medium databases, and low to medium traffic web servers."
    ],
    ["Compute optimized",
     "F, Fs, Fsv2, FX",
     "High CPU-to-memory ratio. Good for medium traffic web servers, network appliances, batch processes, and application servers."
    ],
    ["Memory optimized",
     "Esv3, Ev3, Easv4, Eav4, Epdsv5, Epsv5, Ev4, Esv4, Edv4, Edsv4, Ev5, Esv5, Edv5, Edsv5, Easv5, Eadsv5, Mv2, M, DSv2, Dv2",
      "High memory-to-CPU ratio. Great for relational database servers, medium to large caches, and in-memory analytics."
    ],
    ["Storage optimized",
     "Lsv2, Lsv3, Lasv3",
      "High disk throughput and IO ideal for Big Data, SQL, NoSQL databases, data warehousing and large transactional databases."
    ],
    ["GPU",
     "NC, NCv2, NCv3, NCasT4_v3, ND, NDv2, NV, NVv3, NVv4, NDasrA100_v4, NDm_A100_v4",
     "Specialized virtual machines targeted for heavy graphic rendering and video editing, as well as model training and inferencing (ND) with deep learning. Available with single or multiple GPUs."
    ],
    ["High performance compute",
     "HB, HBv2, HBv3, HBv4, HC, HX",
     "Our fastest and most powerful CPU virtual machines with optional high-throughput network interfaces (RDMA)."
    ]
]

series_data = [{'series': i[0], 'type': i[1].split(','), 'description': i[2]} for i in series_data]
series_data = spark.createDataFrame(series_data)

series_data = series_data \
                .withColumn("type", F.explode("type")) \
                .withColumn("type", F.expr("TRIM(type)")) \
                .withColumn("family", F.expr("REGEXP_EXTRACT(type, '^[A-Z]+', 0)")) \
                .withColumn("version_number", F.expr("REGEXP_EXTRACT(type, 'v[0-9]+$', 0)")) \
                .withColumn("version_number", F.expr("CASE WHEN version_number == '' THEN 'v1' ELSE version_number END"))

series_data = series_data \
                .groupBy("family", "version_number") \
                .agg(F.min("series").alias("series")) \
                .join(series_data.groupBy("series").agg(F.max("description").alias("description")), on="series", how="inner")

df = df.join(series_data, on=["family", "version_number"], how="left_outer")

df.write.format("delta").mode("overwrite").save(delta_location + "/silver/vm_skus")

# Gold Layer

For this last step in the data preparation, we want to explode the dataset rows so that there is an individual record for every minute that a VM was running.

In [0]:
activities = spark.read.format("delta").load(delta_location + "/silver/activities")

In [0]:
from datetime import datetime, timedelta
import pyspark.sql.functions as F

dates = activities \
            .agg(F.min("event_timestamp").alias("min"), F.max("event_timestamp").alias("max")) \
            .selectExpr("*", "(TO_UNIX_TIMESTAMP(max) - TO_UNIX_TIMESTAMP(min)) / 60 AS minutes") \
            .collect()

start_date = dates[0][0]
start_date = datetime(start_date.year, start_date.month, start_date.day, start_date.hour, start_date.minute)
minutes = dates[0][2]

observations = spark.range(minutes) \
                    .withColumn("date", F.lit(start_date) + F.expr("MAKE_INTERVAL(0, 0, 0, 0, 0, id)"))


observations.alias("o") \
    .join(activities.filter("operation NOT LIKE '%delete'").alias("a"),
          observations["date"].between(activities["event_timestamp"], activities["event_end"])) \
    .withColumn("JobName", F.expr("REGEXP_REPLACE(REGEXP_REPLACE(a.tags.RunName, '^ADF\_', ''), '\_[a-f0-9\-]{36}$', '')")) \
    .groupBy("o.date", "a.subscriptionId", "a.workspaceName", "a.vmSize", "a.priority", "a.location", "a.tags.ClusterName", "a.tags.JobId",
             "JobName", "a.tags.RunName", "a.tags.DatabricksInstancePoolId") \
    .count() \
    .orderBy("o.date") \
    .write \
    .format("delta") \
    .save(delta_location + "/gold/observations")

## Data Checks

Now that we've built our datasets, let's check them to make sure they conform to our expectations.

In [0]:
df = spark.read.format("delta").load(delta_location + "/bronze/activity_logs")
activities = spark.read.format("delta").load(delta_location + "/silver/activities")
observations = spark.read.format("delta").load(delta_location + "/gold/observations")

print(f"Bronze Rows: {df.count():,}")
print(f"Silver Rows: {activities.count():,}")
print(f"Gold Rows:   {observations.count():,}")

Bronze Rows: 16,790
Silver Rows: 4,594
Gold Rows:   11,606


In [0]:
print(df.filter("event_timestamp IS NULL").count())
print(df.filter("vm_id IS NULL").count())
print(df.filter("lower(vm_id) NOT rlike '^[0-9a-f][0-9a-f][0-9a-f][0-9a-f][0-9a-f][0-9a-f][0-9a-f][0-9a-f][0-9a-f][0-9a-f][0-9a-f][0-9a-f][0-9a-f][0-9a-f][0-9a-f][0-9a-f][0-9a-f][0-9a-f][0-9a-f][0-9a-f][0-9a-f][0-9a-f][0-9a-f][0-9a-f][0-9a-f][0-9a-f][0-9a-f][0-9a-f][0-9a-f][0-9a-f][0-9a-f][0-9a-f]$'").count())

0
0
0


In [0]:
display(df.groupBy("log_entry.properties.statusCode").count())

statusCode,count
null,14781
Created,997
OK,6
Accepted,1005
Conflict,1


In [0]:
display(df.groupBy("log_entry.operation_name.value", "log_entry.properties.statusCode").count().orderBy("value", "statusCode"))

value,statusCode,count
Microsoft.Compute/virtualMachines/delete,null,3589
Microsoft.Compute/virtualMachines/delete,Accepted,989
Microsoft.Compute/virtualMachines/evictSpotVM/action,null,2
Microsoft.Compute/virtualMachines/write,null,3008
Microsoft.Compute/virtualMachines/write,Accepted,16
Microsoft.Compute/virtualMachines/write,Conflict,1
Microsoft.Compute/virtualMachines/write,Created,997
Microsoft.Compute/virtualMachines/write,OK,6
Microsoft.Resourcehealth/healthevent/Activated/action,null,708
Microsoft.Resourcehealth/healthevent/InProgress/action,null,15


With Azure, there is some time between when a `DELETE` operation request is received and when it completes.  This will help us see the distribution
of minutes it takes to delete a VM.

In [0]:
import pyspark.sql.functions as F

display(df.where("log_entry.operation_name.value == 'Microsoft.Compute/virtualMachines/delete'").groupBy("vm_id", "log_entry.operation_name.value").count().orderBy(F.col("count").desc()))

vm_id,value,count
8e73a2f497194b1ab7a2bd01c5a2605a,Microsoft.Compute/virtualMachines/delete,11
915d56c4a7174c2da9f9182263512aae,Microsoft.Compute/virtualMachines/delete,6
db7765434a514c86a049261032d97665,Microsoft.Compute/virtualMachines/delete,5
807ea8ff1c3f45479a009efa3ef34a51,Microsoft.Compute/virtualMachines/delete,5
cc253e37cb6b43d8953520ced0913a57,Microsoft.Compute/virtualMachines/delete,5
aac3c4ca7621493187a8cf881b05ce9b,Microsoft.Compute/virtualMachines/delete,5
95d13f7860bd4b28abbd0b03a684e4f3,Microsoft.Compute/virtualMachines/delete,5
d598e385aeaf496e8ecde33b94adbbd4,Microsoft.Compute/virtualMachines/delete,5
d9e8b911ce024226bb6d69021295268b,Microsoft.Compute/virtualMachines/delete,5
4e85eafffcbf477ca9bca2d01d4fe0ee,Microsoft.Compute/virtualMachines/delete,5


This is a quick check to see if there are any Databricks Instance Pools in use.

In [0]:
display(
    df
        .filter("log_entry.properties.responseBody.tags.DatabricksInstancePoolId IS NOT NULL")
        .groupBy("log_entry.properties.responseBody.tags.DatabricksInstancePoolId")
        .agg(F.count("vm_id"), F.min("event_timestamp"), F.max("event_timestamp"))
    )

DatabricksInstancePoolId,count(vm_id),min(event_timestamp),max(event_timestamp)
0405-174049-deter56-pool-kuok9x2a,8,2023-11-07T16:23:25.999638Z,2023-11-07T21:52:09.505633Z


In [0]:
display(activities.groupBy("tags.ClusterName").count())

ClusterName,count
job-357805320464384-run-321506695202488-Job_cluster,5
job-357805320464384-run-771913562678391-Job_cluster,4
job-614481705740581-run-444619923291570-Job_cluster,15
job-614481705740581-run-970588044764242-Job_cluster,12
job-614481705740581-run-983543224773264-Job_cluster,14
job-614481705740581-run-961768226312407-Job_cluster,14
job-614481705740581-run-471536407501924-Job_cluster,13
job-614481705740581-run-745285628543497-Job_cluster,14
job-614481705740581-run-278334351961828-Job_cluster,11
job-614481705740581-run-150062578851439-Job_cluster,14


Here we'll take a quick peek at the information we came here for... how many VM's of each type are in use at any given time.

In [0]:
display(observations.groupBy("vmSize", "date").agg(F.sum("count").alias("count")).orderBy("date"))

vmSize,date,count
Standard_D3_v2,2023-11-07T16:10:00Z,3
Standard_D3_v2,2023-11-07T16:11:00Z,3
Standard_D3_v2,2023-11-07T16:12:00Z,3
Standard_D3_v2,2023-11-07T16:13:00Z,3
Standard_D3_v2,2023-11-07T16:14:00Z,3
Standard_D3_v2,2023-11-07T16:15:00Z,3
Standard_D3_v2,2023-11-07T16:16:00Z,3
Standard_D3_v2,2023-11-07T16:17:00Z,3
Standard_D3_v2,2023-11-07T16:18:00Z,3
Standard_DS4_v2,2023-11-07T16:19:00Z,3


In [0]:
display(observations.groupBy("JobName").agg(F.sum("count").alias("vms")).orderBy(F.col("vms").desc()))

JobName,vms
jp-adbx-job-quick,7646
short - mid-size cluster,7111
null,1047
short - single-node,95
